<img style="float: left; margin: 30px 15px 15px 15px;" src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTFzQj91sOlkeDFkg5HDbjtR4QJYmLXkfMNig&usqp=CAU" width="400" height="500" /> 
    
    
### <font color='navy'> Modelos no lineales para pronósticos. 

**Nombres:** José Manuel Haces López y Rafael Juarez Badillo Chávez

**Fecha:** 16 de Febrero del 2023.

**Expediente**: 734759 y 733240
    
**Profesor:** Oscar David Jaramillo Zuluaga.
    
**Link Github**: https://github.com/JManuelHaces/Tarea3_Haces_Juarez

# Tarea 3: Clase 7

## Enunciado de la Tarea 
- Condensar todos los procedimientos realizados en esta clase donde se implementen los métodos necesarios para crear un correcto flujo de lectura de datos, análisis, ajuste de parámetros, predicción y métricas de errores para un conjunto de datos dado. Algo como lo siguiente:

Código de solución estudiante 1

In [ ]:
# Librerías
import datetime
import warnings
import numpy as np
import pandas as pd
from statistics import mean
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
warnings.filterwarnings('ignore')

# Clase para un Pronóstico Lineal
class LinearForecast:
    # Constructor de la clase
    def __init__(self, time_series, horizonte: int = 10):
        """
        Constructor de la clase
        :param time_series: Df con la serie de tiempo a usar
        """
        self.time_series = time_series
        self.horizonte = horizonte

    @staticmethod
    def plot_ts(data, figsize: tuple = (8, 5)):
        """
        Método para graficar la serie de tiempo
        :param data: Serie de tiempo a usar
        :param figsize: Tamaño de la figura generada
        :return: Gráfica generada
        """
        fig = data.plot(figsize=figsize)
        plt.show()
        return fig

    # Método para diferenciar la serie
    def diff_ts(self, num_diff : int = 0):
        """
        Método para diferenciar la serie el número de veces que se selecciona
        :param num_diff: Número de veces a diferenciar
        :return: # Serie diferenciada
        """
        # Creando la serie
        data = self.time_series
        # Ciclo para diferenciar el número de veces seleccionado
        for i in range(num_diff):
            # Diferenciando la serie y quitando nan
            data = data.diff().dropna()
        # Regresando serie diferenciada
        return data

    # Método para hacer el test 'adf_test'
    @staticmethod
    def adf_test(data):
        """
        Método para realizar el test Dickey-Fuller
        :param data: Serie de tiempo a usar
        :return: Nos regresa las estadísticas y resultados del test Dickey-Fuller,
        así como si la serie es estacionaria o no.
        """
        # Imprimiendo el título
        print("Resultados del Test Dickey-Fuller:")
        # Haciendo el test ADFuller
        dftest = adfuller(data, autolag="AIC")
        # Creando un vector con los valores de cada resultado
        dfoutput = pd.Series(
            dftest[0:4],
            index=[
                "Test Estadístico",
                "P-value",
                "Núm. Lags Usados",
                "Núm. Observaciones Usadas",
            ],
        )
        # Poniendo los valores resultantes en el vector
        for key, value in dftest[4].items():
            dfoutput["Valor Crítico (%s)" % key] = value
        # Imprimiendo el vector completo
        print(dfoutput)
        # Checando si es estacionario o no con un umbral de 5% == 0.05
        if (dftest[1] <= 0.05) & (dftest[4]['5%'] > dftest[0]):
            print("\u001b[32mStationary\u001b[0m")
        else:
            print("\x1b[31mNon-stationary\x1b[0m")

    # Método para plotear gráficas ACF y PACF
    @staticmethod
    def plot_acf_pacf(data, kwargs=dict()):
        """
        Método para graficar las gráficas de ACF y PACF
        - ACF = Autocorrelation Function
        - PACF = Partial Autocorrelation Function
        :param data: Serie de tiempo a usar
        :param kwargs: Agregando kwargs para las gráficas, normalmente para lags
        :return: Regresando la figura generada
        """
        # Creando objeto de ploteo
        fig = plt.figure(figsize=(8,5))
        # Agregando la primera gráfica con Autocorrelación
        ax1 = fig.add_subplot(121)
        plot_acf(data, zero=False, ax=ax1, **kwargs)
        # Agregando la segunda gráfica con Autocorrelación Parcial
        ax2 = fig.add_subplot(122)
        plot_pacf(data, zero=False, ax=ax2, method='ols', **kwargs)
        # Mostrando
        plt.show()

        return fig

    @staticmethod
    def split_dataset(data, train_size: float = 0.8):
        """
        Función que se encarga de dividir la data ingresada en datos de entrenamiento y prueba.
        :param data: Serie de tiempo a usar
        :param train_size: Porcentaje de tamaño del conjunto de train [0, 1]
        :return conjuntos de train y test
        """
        # Dividiendo en Train y Test
        train, test= np.split(data, [int(train_size * len(data))])
        # Regresando los dos conjuntos
        return train, test

    # Método para entrenar un Modelo STL
    @staticmethod
    def stl_model(data, period: int = 12, seas: int = 3, seas_deg: int = 2):
        """
        Método para modelar un modelo STL (Seasonal Trend Decomposition)
        :param data: Serie de Tiempo a usar
        :param period: El periodo del componente estacional (horas, días, meses, años)
        :param seas: El tamaño de la ventana de Loess
        :param seas_deg: El grado polinomial utilizado por Loess
        :return: Modelo entrenado
        """
        # Creando el modelo
        model_stl = STL(data, period = period, seasonal=seas, seasonal_deg=seas_deg)
        # Regresando el modelo
        return model_stl

    # Método para entrenar un Modelo ARIMA
    @staticmethod
    def arima_model(data, p: int = 0, d: int = 0, q: int = 0):
        """
        Método para generar un modelo ARIMA
        :param data: Serie de Tiempo para el Modelo
        :param p: Componente 'p'
        :param d: Componente 'd'
        :param q: Componente 'q'
        :return:
        """
        # Creando el objeto ARIMA con los componentes estacionales establecidos
        arima_model = ARIMA(data, order=(p, d, q))
        # Regresando en el modelo
        return arima_model


    # Método para entrenar un Modelo SARIMAX
    @staticmethod
    def sarimax_model(data, p: int = 0, d: int = 0, q: int = 0, P: int = 0, D: int = 0, Q: int = 0, S: int = 12):
        """
        Método para generar el modelo SARMIMAX
        :param data:
        :param p: Componente 'q'
        :param d: Componente 'd'
        :param q: Componente 'q'
        :param P: Componente Estacional 'P'
        :param D: Componente Estacional 'D'
        :param Q: Componente Estacional 'Q'
        :param S: Estacionalidad 'S'
        :return:
        """
        # Haciendo el modelo con la parte no estacional (p, d, q) y estacional (P, D, Q, S)
        model_sarimax = sm.tsa.statespace.SARIMAX(data, order=(p, d, q), seasonal_order=(P, D, Q, S))
        # Regresando el modelo generado
        return model_sarimax


    @staticmethod
    def evaluate_forecasts(real, predic):
        """
        Función que se encargará de calcular las métricas de error. (MSE, RMSE, MAD, MAPE)
        :param real: Vector con los datos reales
        :param predic: Vector con los datos predichos por el modelo
        """
        # Métrica MSE
        mse = mean_squared_error(real, predic)
        # Métrica RMSE
        rmse = mean_squared_error(real, predic, squared=False)
        # Métrica MAD
        mad = mean([abs(real[i] - pred[i]) for i in range(len(real))])
        # Métrica MAPE
        mape = mean_absolute_percentage_error(real, pred)

        # Regresando las métricas
        return mse, rmse, mad, mape

## Modelo SARIMAX

In [ ]:
# Url de la serie de tiempo
url = 'https://github.com/quaesito/time-series-forecast-sarimax-prophet/blob/master/A4.xlsx?raw=true'

# Leyendo el archivo
df = pd.read_excel(url)

df.head()

In [ ]:
# Tomando las columnas necesarias
df = df[['date', 'hour', 'Energy (kWh)']]

# Ciclo para agregar la hora al datetime
data = pd.DataFrame()
for i in df.index:
    temp = df[df.index == i]
    date = temp['date'] + datetime.timedelta(hours=int(temp['hour'][i]))
    temp['date'] = date
    data = pd.concat([data, temp], axis=0)

# Haciendo el índice
data.index = data['date']
data = data[['Energy (kWh)']]

data.head()

In [ ]:
# Graficando la serie
LinearForecast(time_series=data, horizonte=10).plot_ts(data)

In [ ]:
# Haciendo el test
LinearForecast(time_series=data, horizonte=10).adf_test()

In [ ]:
# Diferenciando y haciendo el test
data = LinearForecast.diff_ts(num_diff=2)
LinearForecast(time_series=data, horizonte=10).adf_test()

In [ ]:
# Ploteando correlaciones
LinearForecast(time_series=data, horizonte=10).plot_acf_pacf(data, {lags=40})

In [ ]:
# Dividiendo en train y test
train, test = plit_dataset(data, train_size=0.8)

# Longitud de test
len(test)

In [ ]:
# Haciendo un modelo Sarimax
model_sarimax = LinearForecast(time_series=data, horizonte=10).sarimax_model(train, 
                                                             p=3, d=2, q=3, 
                                                             P=1, D=2, Q=2, S=24)
# Entrenando el modelo
model_sarimax.fit()

# Haciendo predicciones
pred = model_sarimax.predict(start=test.index.min(), end=test.index.max())

In [ ]:
# Evaluando desempeño
LinearForecast(time_series=data, horizonte=10).evaluate_forecasts(test, pred)

Nota: Ya no alcancé a correr el código final.